In [1]:
%load_ext autoreload
%autoreload 2
    
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # set your cuda device
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import torch
import tla
from tla import utils
from transformers import AutoModelForCausalLM, AutoTokenizer, LogitsProcessorList

device = 'cuda'

In [2]:
tokenizer = AutoTokenizer.from_pretrained("allenai/tulu-2-7b")
model = AutoModelForCausalLM.from_pretrained("allenai/tulu-2-7b").to(device)

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/bazaluk/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
vocab_size = tokenizer.vocab_size

dfa = {
    "edges": [
    (0, 2, utils.populate_edge(['X', 'Y'], vocab_size, tokenizer)),
    (0, 1, utils.populate_edge(['V'], vocab_size, tokenizer)),
    (1, 2, utils.populate_edge(['1', '2', '3', '4', '5'], vocab_size, tokenizer)),
    (2, 3, utils.populate_edge(['->'], vocab_size, tokenizer)),
    (3, 4, utils.populate_edge(['V'], vocab_size, tokenizer)),
    (3, 5, utils.populate_edge(['X','Y'], vocab_size, tokenizer)),
    (4, 5, utils.populate_edge(['1', '2', '3', '4', '5'], vocab_size, tokenizer)),
    (5, 0, utils.populate_edge([','], vocab_size, tokenizer)),
    ],
    "initial_state": 0,
    "accept_states": set([5]),
    "current_state": 0,
}

In [4]:
import pandas as pd
data = pd.read_csv('/media/data/bazaluk/ctrlg_tulu2/data.csv')
data['pred_graphs'] = ''

prefix = data['prefix'].iloc[100]
d_prompt = data['prompt'].iloc[100]
#prefix='output a sequence of "A"s: '
#d_prompt='output one sequence with the letter "A". For example: "AAAA"'
suffix = '</s>'
soft_constraint = '' # use empty string for no soft constraint
prompt = f'<|user|>\n"{d_prompt}<|endoftext|>"{soft_constraint}:\n{prefix}\n<|assistant|>\n'


prefix_ids = tokenizer.encode(prefix)[1:]
suffix_ids = tokenizer.encode(suffix)[1:]
prompt_ids = tokenizer.encode(prompt)

In [9]:
custom_logits_processor = utils.CustomLogitsProcessor(dfa)

min_new_tokens=1
max_new_tokens=40
eos_token_id = tokenizer.encode('<\s>')[1]
# set the hmm_batch_size & temperature
beam_size = 8 # sample 128 sequences
temperature = 0.7

# generate with sampling, temperature=0.7
input_ids = torch.tensor([prompt_ids], device=device)
outputs = model.generate(
        input_ids=input_ids, do_sample=True,
        num_return_sequences=beam_size, 
        min_new_tokens=min_new_tokens, max_new_tokens=max_new_tokens,
        logits_processor=LogitsProcessorList([custom_logits_processor]),
        pad_token_id=eos_token_id,
        renormalize_logits=True,
    )

generated_ids = tla.extract_generated_ids(outputs.tolist(), prompt_ids, suffix_ids, eos_token_id)
tokenizer.decode(generated_ids[0])
#tokenizer.decode(generated_ids[1])

'V1 -> X1 , , V3 -> X'

In [14]:
data['step1'].iloc[100]

'V1->V3,V1->X,X->Y,V3->Y'

In [18]:
dfa

{'edges': [(0, 2, array([False, False, False, ..., False, False, False])),
  (0, 1, array([False, False, False, ..., False, False, False])),
  (1, 2, array([False, False, False, ..., False, False, False])),
  (2, 3, array([False, False, False, ..., False, False, False])),
  (3, 4, array([False, False, False, ..., False, False, False])),
  (3, 5, array([False, False, False, ..., False, False, False])),
  (4, 5, array([False, False, False, ..., False, False, False])),
  (5, 0, array([False, False, False, ..., False, False, False]))],
 'initial_state': 0,
 'accept_states': {5},
 'current_state': 5}

In [8]:
data['prompt'].iloc[1000]

'Extract the causal graph: Identify the causal graph that depicts the relationships in the scenario. Use V1 to represent gender. Use X to represent smoking. Use V3 to represent tar deposit. Use Y to represent lung cancer. The diagram should simply consist of edges denoted in "var1 -> var2" format, separated by commas.'